<a href="https://colab.research.google.com/github/EdvinsVormsbehers/EDIBO_7-PNG-file-in-Python/blob/main/Edv%C4%ABns_EDIBO7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

EDIBO Lesson 7 PNG file in Python

version 1

In [ ]:
import png

width = 255
height = 255
img = []
for y in range(height):
    row = ()
    for x in range(width):
        row = row + (x, max(0, 255 - x - y), y)
    img.append(row)
with open('gradient.png', 'wb') as f:
    w = png.Writer(width, height, greyscale=False)
    w.write(f, img)